In [29]:

import time
import importlib
import math

import cv2 as cv
import numpy as np
import mediapipe as mp

import sched, time

import utils
importlib.reload(utils)



<module 'utils' from 'c:\\Users\\chris\\source\\repos\\behaviour_recognition\\behaviour_recognition\\utils.py'>

### Constants

In [30]:
mp_face_mesh = mp.solutions.face_mesh
FONTS =cv.FONT_HERSHEY_COMPLEX
closed_eyes_frame = 1
yawn_frame = 30


### Variables

In [31]:
frame_counter = 0
closed_eyes_counter = 0
total_blinks = 0
total_yawns = 0

yawn_counter = 0



In [12]:
# landmark detection function 
def landmarksDetection(img, results, draw=False):
    img_height, img_width= img.shape[:2]
    # list[(x,y), (x,y)....]
    mesh_coord = [(int(point.x * img_width), int(point.y * img_height)) for point in results.multi_face_landmarks[0].landmark]
    if draw :
        [cv.circle(img, p, 2, utils.GREEN, -1) for p in mesh_coord]

    # returning the list of tuples for each landmarks 
    return mesh_coord

In [32]:
# face bounder indices 
FACE_OVAL=[ 10, 338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288, 397, 365, 379, 378, 400, 377, 152, 148, 176, 149, 150, 136, 172, 58, 132, 93, 234, 127, 162, 21, 54, 103,67, 109]

# lips indices for Landmarks
LIPS=[ 61, 146, 91, 181, 84, 17, 314, 405, 321, 375,291, 308, 324, 318, 402, 317, 14, 87, 178, 88, 95,185, 40, 39, 37,0 ,267 ,269 ,270 ,409, 415, 310, 311, 312, 13, 82, 81, 42, 183, 78 ]
LOWER_LIPS =[61, 146, 91, 181, 84, 17, 314, 405, 321, 375, 291, 308, 324, 318, 402, 317, 14, 87, 178, 88, 95]
UPPER_LIPS=[ 185, 40, 39, 37,0 ,267 ,269 ,270 ,409, 415, 310, 311, 312, 13, 82, 81, 42, 183, 78] 
# Left eyes indices 
LEFT_EYE =[ 362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385,384, 398 ]
LEFT_EYEBROW =[ 336, 296, 334, 293, 300, 276, 283, 282, 295, 285 ]

# right eyes indices
RIGHT_EYE=[ 33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161 , 246 ]  
RIGHT_EYEBROW=[ 70, 63, 105, 66, 107, 55, 65, 52, 53, 46 ]

## Blink calculator 

In [33]:
def euclaideanDistance(point, point1):
    x, y = point 
    x1, y1 = point1
    distance = math.sqrt((x1-x)**2 + (y1-y)**2)
    return distance 

def blinkRatio(img, landmarks, right_indices, left_indices):
    
    # RIGHT_EYE
    # horizontal line
    rh_right = landmarks[right_indices[0]]
    rh_left = landmarks[right_indices[8]]
    # vertical line
    rv_top = landmarks[right_indices[12]]
    rv_bottom = landmarks[right_indices[4]]

    # LEFT_EYE
    # horizontal line
    lh_right = landmarks[right_indices[0]]
    lh_left = landmarks[right_indices[8]]
    # vertical line
    lv_top = landmarks[11]
    lv_bottom = landmarks[16]

    rh_distance = euclaideanDistance(rh_right, rh_left)
    rv_distance = euclaideanDistance(rv_top, rv_bottom)
    lv_distance = euclaideanDistance(lv_top, lv_bottom)
    lh_distance = euclaideanDistance(lh_right, lh_left)

    right_eye_ratio = rh_distance / rv_distance
    left_eye_ratio = lh_distance / lv_distance

    ratio = (right_eye_ratio + left_eye_ratio) / 2
    
    cv.line(frame, rh_right, rh_left, utils.GREEN, 2)
    cv.line(frame, rv_top, rv_bottom, utils.GREEN, 2)
    return ratio

def yawnRatio(img, landmarks, lips_indices):
    
    # mouth
    # horizontal line
    lip_right = landmarks[lips_indices[0]]
    lip_left = landmarks[lips_indices[10]]
    # vertical line
    lip_top = landmarks[lips_indices[16]]
    lip_bottom = landmarks[lips_indices[34]]

    lip_horizontal_distance = euclaideanDistance(lip_right, lip_left)
    lip_vertical_distance = euclaideanDistance(lip_top, lip_bottom) if euclaideanDistance(lip_top, lip_bottom) else 0.0001


    lip_ratio = lip_horizontal_distance / lip_vertical_distance
    
    #ratio = (right_eye_ratio + left_eye_ratio) / 2
    
    cv.line(frame, lip_right, lip_left, utils.GREEN, 2)
    cv.line(frame, lip_top, lip_bottom, utils.GREEN, 2)
    return lip_ratio    


In [37]:
# Connect to cam
cap = cv.VideoCapture(0)

with mp_face_mesh.FaceMesh(
    max_num_faces=1, 
    refine_landmarks=True, 
    min_detection_confidence=0.5, 
    min_tracking_confidence=0.5
) as face_mesh:

    # starting time here 
    start_time = time.time()

    while cap.isOpened():
        frame_counter +=1 # frame counter
        ret, frame = cap.read()
        if not ret:
            break
        # Show image
        frame = cv.flip(frame, 1)
        rgb_frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        img_height, img_width = frame.shape[:2]   # This is the actual measurements of the frame size.  We'll use this to multiply by the normalised x,y coordinates from results.multi_face_landmarks
        results = face_mesh.process(rgb_frame)
        if results.multi_face_landmarks:
            mesh_coords = landmarksDetection(frame, results, False)
            #print(mesh_coords[p] for p in RIGHT_EYE)
            
            ## Blink
            ratio_eyes = blinkRatio(frame, mesh_coords, RIGHT_EYE, LEFT_EYE)
            cv.putText(frame, f'ratio {round(ratio_eyes,2)}', (100,100), FONTS, 0.6, utils.GREEN, 1)
            ## Blink counter logic
            if ratio_eyes > 5.0:
                cv.putText(frame, 'Blink', (200,30), FONTS, 1.3, utils.RED, 2)
                closed_eyes_counter += 1
            else:
                if closed_eyes_counter > closed_eyes_frame:
                    total_blinks += 1
                    closed_eyes_counter = 0
            cv.putText(frame, f'Total blinks {total_blinks}', (100,150), FONTS, 0.6, utils.BLACK, 1)


            ## Yawn
            ratio_mouth = yawnRatio(frame, mesh_coords, LIPS)
            cv.putText(frame, f'ratio {round(ratio_mouth,2)}', (250,100), FONTS, 0.6, utils.BLACK, 1)
            if ratio_mouth < 1.5:
                cv.putText(frame, 'YAWN', (200,30), FONTS, 1.3, utils.GREEN, 2)
                yawn_counter += 1
            else:
                if yawn_counter > yawn_frame:
                    total_yawns += 1
                    yawn_counter = 0
            cv.putText(frame, f'Total yawns {total_yawns}', (350,150), FONTS, 0.6, utils.BLACK, 1)
       
            
            
            ## calculating  frame per seconds FPS
            end_time = time.time()-start_time
            fps = frame_counter/end_time
            frame =utils.textWithBackground(frame,f'FPS: {round(fps,1)}',FONTS, 1.0, (20, 50), bgOpacity=0.9, textThickness=2)


        cv.imshow('Webcam', frame)

        # check if any keys are being pressed, if this is 'q' break out of while loop and release webcam & close frame
        if cv.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv.destroyAllWindows()

In [40]:
import threading

def calcIt():
    for i in range(25000000):
        pass

def callCalcIt():
    thread = threading.Thread(target=calcIt)
    thread.start()
    while thread.is_alive():
        print('...running')

callCalcIt()

...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running
...running

In [39]:
def calc_it():
    r = 25000000
    for x in xrange(r+1):
        if not (x % 1000):  # report every 1000 ticks
            yield x, r

class Monitor(object):
    def call(self):
        for x, r in calc_it():
            print("Done {0} out of {1}".format(x, r))